# **Install all the Dependencies**

In [14]:
from IPython.display import HTML
from base64 import b64encode
def play_video(filename):
  html = ''
  video = open(filename,'rb').read()
  src = 'data:video/mp4;base64,' + b64encode(video).decode()
  html += fr'' % src
  return HTML(html)

In [ ]:
from IPython.display import YouTubeVideo

YOUTUBE_ID = 'Utky2IXgOQI'


[youtube] Extracting URL: https://www.youtube.com/watch?v=Utky2IXgOQI
[youtube] Utky2IXgOQI: Downloading webpage
[youtube] Utky2IXgOQI: Downloading tv client config
[youtube] Utky2IXgOQI: Downloading player 9a279502-main
[youtube] Utky2IXgOQI: Downloading tv player API JSON
[youtube] Utky2IXgOQI: Downloading ios player API JSON
[youtube] Utky2IXgOQI: Downloading m3u8 information


ERROR: [youtube] Utky2IXgOQI: Requested format is not available. Use --list-formats for a list of available formats


## Importing Core Libraries 

In [1]:
import cv2
import torch
import time
import numpy as np
from absl.flags import FLAGS
from deep_sort_realtime.deepsort_tracker import DeepSort
from ultralytics import YOLO
import mediapipe as mp


## Draw Corner-Style Bounding Box

In [2]:
def draw_corner_rect(img, bbox, line_length=30, line_thickness=5, rect_thickness=1,
                     rect_color=(255, 0, 255), line_color=(0, 255, 0)):
    x, y, w, h = bbox
    x1, y1 = x + w, y + h

    if rect_thickness != 0:
        cv2.rectangle(img, bbox, rect_color, rect_thickness)

    # Top Left  x, y
    cv2.line(img, (x, y), (x + line_length, y), line_color, line_thickness)
    cv2.line(img, (x, y), (x, y + line_length), line_color, line_thickness)

    # Top Right  x1, y
    cv2.line(img, (x1, y), (x1 - line_length, y), line_color, line_thickness)
    cv2.line(img, (x1, y), (x1, y + line_length), line_color, line_thickness)

    # Bottom Left  x, y1
    cv2.line(img, (x, y1), (x + line_length, y1), line_color, line_thickness)
    cv2.line(img, (x, y1), (x, y1 - line_length), line_color, line_thickness)

    # Bottom Right  x1, y1
    cv2.line(img, (x1, y1), (x1 - line_length, y1), line_color, line_thickness)
    cv2.line(img, (x1, y1), (x1, y1 - line_length), line_color, line_thickness)

    return img

In [ ]:
SOURCE_VIDEO_PATH = "../data/youtube.mp4"
OUTPUT_VIDEO_PATH = "../data/output.mp4"
BLUR_ID = None
CONF = 0.8
CLASS_ID = None
OUTPUT_CSV_PATH = "../data/player_metrics.csv"

In [4]:
model = YOLO("models/yolov9s.pt") 

## Initializing VideoCapture & VideoWriter with DeepSort Tracking and Device Selection

In [5]:
cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
# video writer objects
fourcc = cv2.VideoWriter_fourcc(*'XVID')
writer = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (frame_width, frame_height))

if not writer.isOpened():
    print("Failed to open VideoWriter!")
    cap.release()
    exit(1)

# Initialize the DeepSort tracker
tracker = DeepSort(max_age=50)
# select device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



## YOLOv9 + DeepSort

# Generate random colors for each class
np.random.seed(42)
colors = np.random.randint(0, 255, size=(len(model.names), 3))

# Parameters
CONF    = 0.8            # confidence threshold
ALLOWED = [0, 32]        # 0=person, 32=sports ball

# Manual ball tracking
ball_id         = 1
last_ball_bbox  = None   # will hold [x, y, w, h] of the ball

# FPS calculation
frame_count = 0
start_time  = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 1. Run detection
    results = model(frame)
    r       = results[0]
    xyxy    = r.boxes.xyxy.cpu().numpy()
    confs   = r.boxes.conf.cpu().numpy()
    cls_ids = r.boxes.cls.cpu().numpy().astype(int)

    # 2. Build filtered detection list
    detect = []
    for (x1, y1, x2, y2), conf, cid in zip(xyxy, confs, cls_ids):
        if conf < CONF or cid not in ALLOWED:
            continue
        detect.append([[int(x1), int(y1), int(x2 - x1), int(y2 - y1)],
                       float(conf), int(cid)])

    # 3. Track persons with DeepSort
    person_detections = [d for d in detect if d[2] == 0]
    tracks            = tracker.update_tracks(person_detections, frame=frame)

    # 4. Manual track for the single sports ball
    ball_candidates = [d for d in detect if d[2] == 32]
    if ball_candidates:
        best_ball       = max(ball_candidates, key=lambda x: x[1])
        last_ball_bbox  = best_ball[0]

    # 5. Draw person tracking results
    for track in tracks:
        if not track.is_confirmed():
            continue

        tid       = track.track_id
        x1, y1, x2, y2 = map(int, track.to_ltrb())
        cid       = track.get_det_class()  # should be 0
        color     = colors[cid]
        B, G, R   = map(int, color)
        label     = f"{tid} - {model.names[cid]}"

        # draw corner-style box
        frame = draw_corner_rect(
            frame,
            (x1, y1, x2 - x1, y2 - y1),
            line_length=15,
            line_thickness=3,
            rect_thickness=1,
            rect_color=(B, G, R),
            line_color=(R, G, B),
        )
        # draw solid rectangle and label background
        cv2.rectangle(frame, (x1, y1), (x2, y2), (B, G, R), 2)
        cv2.rectangle(
            frame,
            (x1 - 1, y1 - 20),
            (x1 + len(label) * 10, y1),
            (B, G, R),
            -1
        )
        cv2.putText(
            frame,
            label,
            (x1 + 5, y1 - 7),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (255, 255, 255),
            2
        )

    # 6. Draw the sports ball with fixed ID
    if last_ball_bbox is not None:
        x, y, w, h = last_ball_bbox
        color       = colors[32]
        B, G, R     = map(int, color)
        label       = f"{ball_id} - sports ball"

        frame = draw_corner_rect(
            frame,
            (x, y, w, h),
            line_length=15,
            line_thickness=3,
            rect_thickness=1,
            rect_color=(B, G, R),
            line_color=(R, G, B),
        )
        cv2.rectangle(frame, (x, y), (x + w, y + h), (B, G, R), 2)
        cv2.rectangle(frame, (x - 1, y - 20), (x + len(label) * 10, y), (B, G, R), -1)
        cv2.putText(frame, label, (x + 5, y - 7),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # 7. Write frame and update FPS
    writer.write(frame)
    frame_count += 1
    if frame_count % 10 == 0:
        fps_calc = frame_count / (time.time() - start_time)
        print(f"FPS: {fps_calc:.2f}")

    # Exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources after loop
cap.release()
writer.release()
cv2.destroyAllWindows()


## YOLOv9 + DeepSort + MediaPipe

In [ ]:
import numpy as np

def assess_speed_quality(speeds_pps, px_to_m):
    """
    Evaluates player speed quality based on average speed in m/s.

    Args:
        speeds_pps: list of floats, speeds in pixels per second.
        px_to_m: float, conversion factor from pixel to meter (m/px).

    Returns:
        dict with:
            - 'average_speed_mps': average speed in meters/second
            - 'max_speed_mps': maximum speed in meters/second
            - 'quality': one of 'Excellent Speed', 'Good Speed', 'Poor Speed'
            - 'description': English description of the quality
    """
    # Convert to m/s
    valid_pps = [s for s in speeds_pps if s is not None and s > 0]
    speeds_mps = [s * px_to_m for s in valid_pps] if valid_pps else [0.0]
    
    average_speed_mps = float(np.mean(speeds_mps))
    max_speed_mps = float(np.max(speeds_mps))
    
    # Classify quality based on average speed
    if average_speed_mps >= 6.0:
        quality = 'Excellent Speed'
        description = (
            'Player demonstrates very high average speed (>=6 m/s), '
            'able to outrun opponents and cover ground effectively.'
        )
    elif average_speed_mps >= 4.0:
        quality = 'Good Speed'
        description = (
            'Player maintains solid speed (4–6 m/s) in most game situations, '
            'supporting both offensive runs and defensive recoveries.'
        )
    else:
        quality = 'Poor Speed'
        description = (
            "Player's average speed (<4 m/s) is below desired levels, "
            'which may limit ability to keep up with play pace.'
        )
    
    return {
        'average_speed_mps': average_speed_mps,
        'max_speed_mps': max_speed_mps,
        'quality': quality,
        'description': description
    }

# Example usage:
# px_to_m = 0.02
# result = assess_speed_quality(speed_history[player_id], px_to_m)
# print(result)




In [ ]:
import math
import numpy as np

def assess_ball_control_quality(keypoints_history, ball_history, px_to_m, foot_indices=(27, 28)):
    """
    Calculates average distance between the ball and player's feet (in meters), then categorizes control quality.

    Returns:
        dict with:
        - 'avg_distance_m': average distance in meters
        - 'quality': one of 'Excellent Control', 'Good Control', 'Poor Control'
        - 'description': English description of the control quality
    """
    distances_m = []
    n = min(len(keypoints_history), len(ball_history))
    
    for i in range(n):
        kps = keypoints_history[i]
        ball = ball_history[i]
        if ball is None or not kps:
            continue
        
        # Calculate minimum distance in pixels between ball and any foot
        pts = [kps[idx] for idx in foot_indices if idx < len(kps)]
        dists_px = [math.hypot(ball[0] - px, ball[1] - py) for px, py in pts]
        if not dists_px:
            continue
        
        # Convert to meters
        min_dist_m = min(dists_px) * px_to_m
        distances_m.append(min_dist_m)
    
    if not distances_m:
        return {
            'avg_distance_m': None,
            'quality': 'Poor Control',
            'description': 'Insufficient data to assess control quality.'
        }
    
    avg_dist = float(np.mean(distances_m))
    
    # Determine control quality
    if avg_dist < 1.0:
        quality = 'Excellent Control'
        description = (
            'Ball remains very close to the player’s feet (under 1 meter), '
            'allowing quick decision-making for passing, shooting, or dribbling.'
        )
    elif avg_dist < 2.0:
        quality = 'Good Control'
        description = (
            'Ball is slightly farther (1–2 meters), but the player still '
            'manages it comfortably without heavy opponent pressure.'
        )
    else:
        quality = 'Poor Control'
        description = (
            'Ball drifts beyond 2 meters, increasing the risk of losing '
            'possession or reducing attacking effectiveness.'
        )
    
    return {
        'avg_distance_m': avg_dist,
        'quality': quality,
        'description': description
    }


In [ ]:
import math
import numpy as np

def assess_self_passes(keypoints_history, ball_history, px_to_m,
                       foot_indices=(27, 28),
                       control_thresh_m=0.5):
    """
    Detects self-pass events where the player kicks the ball and then regains control.
    Classifies each event based on the distance traveled by the ball before returning.

    Args:
        keypoints_history: list of lists of (x,y) pixel coords per frame
        ball_history: list of (x,y) pixel coords or None per frame
        px_to_m: conversion factor from pixel to meter (m/px)
        foot_indices: indices for foot landmarks in MediaPipe
        control_thresh_m: distance in meters defining control proximity

    Returns:
        List of dicts, each with:
            - 'distance_m': float, max distance ball traveled before return
            - 'quality': str category ('Excellent Self Pass', 'Effective Self Pass', 'Poor Self Pass')
            - 'description': English description of quality
            - 'start_frame': frame index where pass started
            - 'end_frame': frame index where pass ended
    """
    control_thresh_px = control_thresh_m / px_to_m
    events = []
    in_pass = False
    start_frame = None
    max_dist_px = 0
    
    n = min(len(keypoints_history), len(ball_history))
    
    for i in range(n):
        kps = keypoints_history[i]
        ball = ball_history[i]
        if ball is None or not kps:
            # Reset if lost tracking
            if in_pass:
                in_pass = False
                start_frame = None
                max_dist_px = 0
            continue
        
        # Compute foot centroid
        feet = [kps[idx] for idx in foot_indices if idx < len(kps)]
        if not feet:
            continue
        fx = sum(p[0] for p in feet) / len(feet)
        fy = sum(p[1] for p in feet) / len(feet)
        
        dist_px = math.hypot(ball[0] - fx, ball[1] - fy)
        
        if not in_pass:
            # Begin pass when ball leaves control area
            if dist_px > control_thresh_px:
                in_pass = True
                start_frame = i
                max_dist_px = dist_px
        else:
            # Update max distance
            if dist_px > max_dist_px:
                max_dist_px = dist_px
            # End pass when ball returns to control area
            if dist_px <= control_thresh_px:
                end_frame = i
                # Convert max distance to meters
                distance_m = max_dist_px * px_to_m
                # Classify
                if 2 <= distance_m < 5:
                    quality = 'Excellent Self Pass'
                    description = (
                        'Player pushes the ball 2–5 meters ahead into space, '
                        'bypassing opponents and regaining it without defensive intervention. '
                        'Ideal for dribbling or speeding up play.'
                    )
                elif 5 <= distance_m < 8:
                    quality = 'Effective Self Pass'
                    description = (
                        'Ball travels 5–8 meters, requiring speed and response to chase. '
                        'Used for long passes to bypass defenders or quick wing play; '
                        'successful if no defender intercepts.'
                    )
                else:
                    quality = 'Poor Self Pass'
                    description = (
                        'Ball is sent over 8 meters, increasing interception risk '
                        'or loss of control. Often deemed a misplaced or overly strong pass.'
                    )
                events.append({
                    'distance_m': distance_m,
                    'quality': quality,
                    'description': description,
                    'start_frame': start_frame,
                    'end_frame': end_frame
                })
                # Reset for next pass
                in_pass = False
                start_frame = None
                max_dist_px = 0
    
    return events

# Example usage:
# px_to_m = 0.02  # conversion factor
# events = assess_self_passes(keypoints_history[player_id], ball_history, px_to_m)
# print(events)


In [ ]:
import pandas as pd

def export_player_metrics_to_csv(speed_history, keypoints_history, ball_history, px_to_m, output_path):
    """
    Aggregates player performance metrics and writes them to a CSV file.

    Args:
        speed_history (dict): {player_id: [speeds_pps]}
        keypoints_history (dict): {player_id: [[(x,y), ...], ...]}
        ball_history (list): [(x,y) or None] per frame
        px_to_m (float): conversion factor from pixel to meter
        output_path (str): file path for the CSV output

    Returns:
        None
    """
    rows = []
    for player_id, speeds in speed_history.items():
        # Speed assessment
        speed_res = assess_speed_quality(speeds, px_to_m)
        
        # Ball control assessment
        control_res = assess_ball_control_quality(
            keypoints_history.get(player_id, []),
            ball_history,
            px_to_m
        )
        
        # Self-pass events and counts
        self_pass_events = assess_self_passes(
            keypoints_history.get(player_id, []),
            ball_history,
            px_to_m
        )
        count_excellent = sum(1 for e in self_pass_events if e['quality'] == 'Excellent Self Pass')
        count_effective = sum(1 for e in self_pass_events if e['quality'] == 'Effective Self Pass')
        count_poor = sum(1 for e in self_pass_events if e['quality'] == 'Poor Self Pass')
        
        rows.append({
            'player_id': player_id,
            'avg_speed_mps': speed_res['average_speed_mps'],
            'max_speed_mps': speed_res['max_speed_mps'],
            'speed_quality': speed_res['quality'],
            'avg_control_distance_m': control_res['avg_distance_m'],
            'control_quality': control_res['quality'],
            'num_excellent_self_pass': count_excellent,
            'num_effective_self_pass': count_effective,
            'num_poor_self_pass': count_poor
        })

    df = pd.DataFrame(rows)
    df.to_csv(output_path, index=False)
    print(f"Player metrics exported to {output_path}")

# Example usage:
# export_player_metrics_to_csv(speed_history, keypoints_history, ball_history, px_to_m, 'player_metrics.csv')


In [ ]:
# MediaPipe Pose
mp_drawing = mp.solutions.drawing_utils
mp_pose    = mp.solutions.pose
pose       = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    enable_segmentation=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# YOLO & DeepSort setup assumed done: model, tracker, cap, writer...
# Generate random colors for each class
np.random.seed(42)
colors = np.random.randint(0, 255, size=(len(model.names), 3))

# Parameters
CONF    = 0.8            # confidence threshold
ALLOWED = [0, 32]        # 0=person, 32=sports ball

# Manual ball tracking
ball_id         = 1
last_ball_bbox  = None   # will hold [x, y, w, h] of the ball

# FPS calculation
frame_count = 0
start_time  = time.time()

# Initialize evaluation histories
speed_history = {}
keypoints_history = {}
ball_history = []
prev_positions = {}

# Main loop
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 1. Detection
    results = model(frame)[0]
    xyxy = results.boxes.xyxy.cpu().numpy()
    confs = results.boxes.conf.cpu().numpy()
    cls_ids = results.boxes.cls.cpu().numpy().astype(int)

    detect = [
        [[int(x1), int(y1), int(x2 - x1), int(y2 - y1)], float(conf), int(cid)]
        for (x1, y1, x2, y2), conf, cid in zip(xyxy, confs, cls_ids)
        if conf >= CONF and cid in ALLOWED
    ]

    # 2. Person tracking
    person_dets = detect = [d for d in detect if d[2] == 0]
    tracks = tracker.update_tracks(person_dets, frame=frame)

    # 3. Ball tracking
    ball_candidates = [d for d in detect if d[2] == 32]
    ball_center = None
    if ball_candidates:
        bx, by, bw, bh = max(ball_candidates, key=lambda x: x[1])[0]
        ball_center = (bx + bw/2, by + bh/2)
    ball_history.append(ball_center)

    # 4. Pose + metrics per person
    for track in tracks:
        if not track.is_confirmed():
            continue
        tid = track.track_id
        x1, y1, x2, y2 = map(int, track.to_ltrb())
        person_roi = frame[y1:y2, x1:x2]
        if person_roi.size == 0:
            continue

        # Pose estimation
        roi_rgb = cv2.cvtColor(person_roi, cv2.COLOR_BGR2RGB)
        res = pose.process(roi_rgb)
        if not res.pose_landmarks:
            continue

        # Convert landmarks to pixel coords
        h_roi, w_roi = person_roi.shape[:2]
        kps = []
        for lm in res.pose_landmarks.landmark:
            px = int(lm.x * w_roi) + x1
            py = int(lm.y * h_roi) + y1
            kps.append((px, py))
        keypoints_history.setdefault(tid, []).append(kps)


        # Calculate speed using hip landmark (24) via prev_positions
        now = time.time()
        hip_x, hip_y = kps[24]  # مفصل الحوض

        if tid in prev_positions:
            x_prev, y_prev, t_prev = prev_positions[tid]
            dt = now - t_prev or 1e-6
            speed = math.hypot(hip_x - x_prev, hip_y - y_prev) / dt
            speed_history.setdefault(tid, []).append(speed)
        else:
            speed_history.setdefault(tid, []).append(0.0)

        # حدّث prev_positions بالموضع والوقت الحاليين
        prev_positions[tid] = (hip_x, hip_y, now)


        # Draw landmarks on ROI
        mp_drawing.draw_landmarks(
            person_roi,
            res.pose_landmarks,
            mp_pose.POSE_CONNECTIONS
        )
        frame[y1:y2, x1:x2] = person_roi

    # 5. Draw detections & tracks...
    # [existing draw_corner_rect and ball code here]

    # 5. Draw person tracking results
    for track in tracks:
        if not track.is_confirmed():
            continue

        tid       = track.track_id
        x1, y1, x2, y2 = map(int, track.to_ltrb())
        cid       = track.get_det_class()  # should be 0
        color     = colors[cid]
        B, G, R   = map(int, color)
        label     = f"{tid} - {model.names[cid]}"

        # draw corner-style box
        frame = draw_corner_rect(
            frame,
            (x1, y1, x2 - x1, y2 - y1),
            line_length=15,
            line_thickness=3,
            rect_thickness=1,
            rect_color=(B, G, R),
            line_color=(R, G, B),
        )
        # draw solid rectangle and label background
        cv2.rectangle(frame, (x1, y1), (x2, y2), (B, G, R), 2)
        cv2.rectangle(
            frame,
            (x1 - 1, y1 - 20),
            (x1 + len(label) * 10, y1),
            (B, G, R),
            -1
        )
        cv2.putText(
            frame,
            label,
            (x1 + 5, y1 - 7),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (255, 255, 255),
            2
        )

    # 6. Draw the sports ball with fixed ID
    if last_ball_bbox is not None:
        x, y, w, h = last_ball_bbox
        color       = colors[32]
        B, G, R     = map(int, color)
        label       = f"{ball_id} - sports ball"

        frame = draw_corner_rect(
            frame,
            (x, y, w, h),
            line_length=15,
            line_thickness=3,
            rect_thickness=1,
            rect_color=(B, G, R),
            line_color=(R, G, B),
        )
        cv2.rectangle(frame, (x, y), (x + w, y + h), (B, G, R), 2)
        cv2.rectangle(frame, (x - 1, y - 20), (x + len(label) * 10, y), (B, G, R), -1)
        cv2.putText(frame, label, (x + 5, y - 7),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # 6. Evaluate and export player metrics to csv 
    px_to_m = 0.02  # e.g. 1 pixel = 0.02 meters
    export_player_metrics_to_csv(
            speed_history,
            keypoints_history,
            ball_history,
            px_to_m,
            OUTPUT_CSV_PATH
        )

    # 7. Write and show
    writer.write(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
writer.release()
cv2.destroyAllWindows()
pose.close()


0: 640x384 1 person, 1 sports ball, 333.3ms
Speed: 2.2ms preprocess, 333.3ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)
Player 1 -> Speed: {'average_speed_pps': 0.0, 'max_speed_pps': 0.0}, Control: 0.00, Juggling: 0
Speed Metrics: {'average_speed_pps': 0.0, 'max_speed_pps': 0.0}
Ball Control: 0.0%
Juggling Touches: 0

0: 640x384 1 person, 1 sports ball, 208.1ms
Speed: 3.4ms preprocess, 208.1ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)
Player 1 -> Speed: {'average_speed_pps': 21.672014144565146, 'max_speed_pps': 43.34402828913029}, Control: 0.00, Juggling: 0
Speed Metrics: {'average_speed_pps': 21.672014144565146, 'max_speed_pps': 43.34402828913029}
Ball Control: 0.0%
Juggling Touches: 0

0: 640x384 1 person, 1 sports ball, 216.7ms
Speed: 3.3ms preprocess, 216.7ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)
Player 1 -> Speed: {'average_speed_pps': 17.74040398834765, 'max_speed_pps': 43.34402828913029}, Control: 0.00, Jug

# MediaPipe Pose
mp_drawing = mp.solutions.drawing_utils
mp_pose    = mp.solutions.pose
pose       = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    enable_segmentation=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Generate random colors for each class
np.random.seed(42)
colors = np.random.randint(0, 255, size=(len(model.names), 3))

# Parameters
CONF    = 0.8            # confidence threshold
ALLOWED = [0, 32]        # 0=person, 32=sports ball

# Manual ball tracking
ball_id         = 1
last_ball_bbox  = None   # will hold [x, y, w, h] of the ball

# FPS calculation
frame_count = 0
start_time  = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 1. Run detection
    results = model(frame)
    r       = results[0]
    xyxy    = r.boxes.xyxy.cpu().numpy()
    confs   = r.boxes.conf.cpu().numpy()
    cls_ids = r.boxes.cls.cpu().numpy().astype(int)

    # 2. Build filtered detection list
    detect = []
    for (x1, y1, x2, y2), conf, cid in zip(xyxy, confs, cls_ids):
        if conf < CONF or cid not in ALLOWED:
            continue
        detect.append([[int(x1), int(y1), int(x2 - x1), int(y2 - y1)],
                       float(conf), int(cid)])

    # 3. Track persons with DeepSort
    person_detections = [d for d in detect if d[2] == 0]
    tracks            = tracker.update_tracks(person_detections, frame=frame)

    # 4. Manual track for the single sports ball
    ball_candidates = [d for d in detect if d[2] == 32]
    if ball_candidates:
        best_ball       = max(ball_candidates, key=lambda x: x[1])
        last_ball_bbox  = best_ball[0]

        # ————————— Pose Estimation داخل كل ROI لشخص مؤكد —
    for track in tracks:
        if not track.is_confirmed():
            continue
        # إحداثيات الـ bounding box
        x1, y1, x2, y2 = map(int, track.to_ltrb())
        # قص المنطقة
        person_roi = frame[y1:y2, x1:x2]
        if person_roi.size == 0:
            continue
        # مرّر ROI للـ Pose model
        roi_rgb = cv2.cvtColor(person_roi, cv2.COLOR_BGR2RGB)
        pose_res = pose.process(roi_rgb)
        # إذا وجد landmarks، ارسمها داخل الـ ROI
        if pose_res.pose_landmarks:
            mp_drawing.draw_landmarks(
                person_roi,
                pose_res.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(thickness=2)
            )
            # اعد الـ ROI المرسوم إلى الفريم الأصلي
            frame[y1:y2, x1:x2] = person_roi


    # 5. Draw person tracking results
    for track in tracks:
        if not track.is_confirmed():
            continue

        tid       = track.track_id
        x1, y1, x2, y2 = map(int, track.to_ltrb())
        cid       = track.get_det_class()  # should be 0
        color     = colors[cid]
        B, G, R   = map(int, color)
        label     = f"{tid} - {model.names[cid]}"

        # draw corner-style box
        frame = draw_corner_rect(
            frame,
            (x1, y1, x2 - x1, y2 - y1),
            line_length=15,
            line_thickness=3,
            rect_thickness=1,
            rect_color=(B, G, R),
            line_color=(R, G, B),
        )
        # draw solid rectangle and label background
        cv2.rectangle(frame, (x1, y1), (x2, y2), (B, G, R), 2)
        cv2.rectangle(
            frame,
            (x1 - 1, y1 - 20),
            (x1 + len(label) * 10, y1),
            (B, G, R),
            -1
        )
        cv2.putText(
            frame,
            label,
            (x1 + 5, y1 - 7),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (255, 255, 255),
            2
        )

    # 6. Draw the sports ball with fixed ID
    if last_ball_bbox is not None:
        x, y, w, h = last_ball_bbox
        color       = colors[32]
        B, G, R     = map(int, color)
        label       = f"{ball_id} - sports ball"

        frame = draw_corner_rect(
            frame,
            (x, y, w, h),
            line_length=15,
            line_thickness=3,
            rect_thickness=1,
            rect_color=(B, G, R),
            line_color=(R, G, B),
        )
        cv2.rectangle(frame, (x, y), (x + w, y + h), (B, G, R), 2)
        cv2.rectangle(frame, (x - 1, y - 20), (x + len(label) * 10, y), (B, G, R), -1)
        cv2.putText(frame, label, (x + 5, y - 7),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # 7. Write frame and update FPS
    writer.write(frame)
    frame_count += 1
    if frame_count % 10 == 0:
        fps_calc = frame_count / (time.time() - start_time)
        print(f"FPS: {fps_calc:.2f}")

    # Exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources after loop
cap.release()
writer.release()
cv2.destroyAllWindows()
pose.close()